In [7]:
# add local directory
import sys
sys.path.append('../')
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/workspace/path_demo
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython
../
../


In [12]:
import os

import torch
import numpy as np

from aad.datasets import DATASET_LIST, DataContainer
from aad.basemodels import ModelContainerPT, MnistCnnCW
from aad.attacks import SaliencyContainer

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
DATA_ROOT = os.path.join('..', 'data')
BATCH_SIZE = 128

# image datasets: {'MNIST', 'CIFAR10', 'SVHN'}
# quantitative datasets: {'BankNote', 'BreastCancerWisconsin', 'HTRU2', 'Iris', 'WheatSeed'}
NAME = 'MNIST'
print(f'Starting {NAME} data container...')
print(DATASET_LIST[NAME])

dc = DataContainer(DATASET_LIST[NAME], DATA_ROOT)
dc(size_train=0.8, normalize=True)

num_features = dc.dim_data[0]
num_classes = dc.num_classes
print('Features:', num_features)
print('Classes:', num_classes)

## model in {BCNN, IrisNN, MnistCnnCW}
# model = IrisNN(num_features, num_classes, hidden_nodes=16)  # for Iris
# model = BCNN(num_features, num_classes)
model = MnistCnnCW()
model_name = model.__class__.__name__
print('Using model:', model_name)

mc = ModelContainerPT(model, dc)
mc.fit(epochs=3, batch_size=BATCH_SIZE)

Starting MNIST data container...
{'name': 'MNIST', 'type': 'image', 'size': 70000, 'num_classes': 10, 'dim_data': (1, 28, 28)}
Loading data...
Preparing DataLoaders...
Preparing numpy arrays...
Successfully load data! Time to complete:  0m 3.8s
Features: 1
Classes: 10
Using model: MnistCnnCW


2020-03-21 05:37:58,769:DEBUG:pt_model_container:Using device: cuda:0


{'lr': 0.1, 'momentum': 0}


2020-03-21 05:38:01,178:INFO:pt_model_container:[ 1/3]  0m 2.4s - Train Loss: 1.2966 Acc: 53.4700% - Test Loss: 0.1077 Acc: 96.6700%
2020-03-21 05:38:03,536:INFO:pt_model_container:[ 2/3]  0m 2.4s - Train Loss: 0.0955 Acc: 97.0067% - Test Loss: 0.0646 Acc: 98.0300%
2020-03-21 05:38:05,897:INFO:pt_model_container:[ 3/3]  0m 2.4s - Train Loss: 0.0591 Acc: 98.1333% - Test Loss: 0.0465 Acc: 98.5200%


Time to complete training:  0m 7.1s


In [15]:
attack = SaliencyContainer(mc)
print(attack.attack_params)

2020-03-21 05:38:13,273:INFO:pytorch:Inferred 1 hidden layers on PyTorch classifier.


(0.0, 1.0)
{'theta': 0.1, 'gamma': 1.0, 'batch_size': 16}


In [16]:
n = 32

# targeted
# targets = attack.randam_targets(n, dc.num_classes, use_onehot=True)
# adv, y_adv, x_clean, y_clean = attack.generate(count=n, targets=targets)

# untargeted
adv, y_adv, x_clean, y_clean = attack.generate(count=n)
print(attack.attack_params)
print(adv.shape)
print(y_adv.shape)
print(x_clean.shape)
print(y_clean.shape)

2020-03-21 05:38:17,204:INFO:saliency_map:Success rate of JSMA attack: 100.00%


Time to complete training 32 adversarial examples:  0m 3.1s
{'theta': 0.1, 'gamma': 1.0, 'batch_size': 16}
(32, 1, 28, 28)
(32,)
(32, 1, 28, 28)
(32,)


In [17]:
accuracy = mc.evaluate(x_clean, y_clean)
print(f'Accuracy on clean samples: {accuracy*100:.4f}%')
accuracy = mc.evaluate(adv, y_clean)
print(f'Accuracy on adversarial example: {accuracy*100:.4f}%')

Accuracy on clean samples: 100.0000%
Accuracy on adversarial example: 0.0000%
